In [65]:
import numpy as np

import qiskit
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, assemble
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import PulseSimulator, Aer, QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.pulse import PulseSystemModel, hamiltonian_model
from qiskit.test.mock import FakeArmonk
from qiskit.tools import job_monitor
from src.helper import qutip_ham_converter
from src.qoc_instruction_schedule_map import QOCInstructionScheduleMap
from src.QutipOptimizer import QutipOptimizer

In [66]:
IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
real_armonk_backend = provider.get_backend('ibmq_armonk')
fake_armonk_backend = FakeArmonk()

ibmqfactory.load_account:WARNING:2020-08-06 13:57:27,793: Credentials are already in use. The existing account in the session will be replaced.


In [67]:
omegad0 = getattr(real_armonk_backend.configuration(), 'hamiltonian')['vars']['omegad0']
omegad0

0.07188793042715262

In [68]:
#set armonk sim parameters
omegad0 = 31919806.545849085

# omegad0 = 0.07
omegad0 = omegad0 * .5 * 1e9
# make sure to print omegad0 and wq0
getattr(fake_armonk_backend.configuration(), 'hamiltonian')['vars']['omegad0'] = omegad0
freq_est = 4.97445401836326e9
fake_armonk_backend.configuration().qubit_freq_est=[freq_est]
getattr(fake_armonk_backend.configuration(), 'hamiltonian')['vars']['wq0'] = 2*np.pi*freq_est
fake_armonk_model = PulseSystemModel.from_backend(fake_armonk_backend)

In [69]:
# fake_armonk_backend=real_armonk_backend

In [70]:
# Instantiate grape optimizer with armonk backend
grape_optimizer = QutipOptimizer(fake_armonk_backend, n_ts=320)

# Create new QocInstructionScheduleMap with this optimizer
def_inst = ['measure']
builtin_instructions = fake_armonk_backend.defaults().instruction_schedule_map
grape_inst_map = QOCInstructionScheduleMap.from_inst_map(grape_optimizer, builtin_instructions,
                                                         default_inst=def_inst)


In [71]:
# choose random phi for u3 rotation
phi = np.random.random_sample(1)
theta = np.random.random_sample(1)
lamb = np.random.random_sample(1)
phi = phi[0] * np.pi
# phi = np.pi

In [72]:
# create circuit
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ = qiskit.QuantumCircuit(q, c)
# add an x gate using u3
circ.x(q)
# circ.h(q)
# circ.u3(phi,0,phi, q)
# circ.u3(theta,phi,lamb, q)
# add measurement pulse
circ.measure([0],[0])

In [73]:
# construct the schedule from the circuit using the grape instruction_schedule_map
grape_schedule = qiskit.schedule(circ, inst_map = grape_inst_map,
                        meas_map = fake_armonk_backend.configuration().meas_map)

gate hit


In [74]:
# now compare to the default
default_schedule = qiskit.schedule(circ, inst_map = builtin_instructions,
                        meas_map = fake_armonk_backend.configuration().meas_map)

changed instruction to instruction.name


In [75]:
grape_schedule.duration
default_schedule.duration

16640

In [76]:
# grape_schedule.draw(plot_range=[0,2000], label=True)

In [77]:
# default_schedule.draw(plot_range=[0,2000], label=True)
# grape_schedule.draw(label=True, scaling=0.8)

In [78]:
# assemble qobj for job submission
backend_sim = PulseSimulator(configuration=fake_armonk_backend.configuration)

grape_qobj = assemble(grape_schedule,
                      backend=backend_sim,
                      qubit_lo_freq=[freq_est],
                      meas_level=2,
                      meas_return='single',
                      shots=1024)

default_qobj = assemble(default_schedule,
                      backend=backend_sim,
                      qubit_lo_freq=[freq_est],
                      meas_level=2,
                      meas_return='single',
                      shots=1024)

In [79]:
sim_result = backend_sim.run(grape_qobj, fake_armonk_model).result()
def_sim_result = backend_sim.run(default_qobj, fake_armonk_model).result()
# sim_result.get_counts()

In [80]:
noise_model = NoiseModel()
noise_model.add_basis_gates(['unitary'])
ideal_job = qiskit.execute(circ, QasmSimulator(),
                           basis_gates=noise_model.basis_gates)
# ideal_job.result().get_counts()



In [81]:
num_shots = 1024

In [82]:
armonk_qobj = assemble(grape_schedule,
                      backend=real_armonk_backend,
                      qubit_lo_freq=[freq_est],
                      meas_level=2,
                      meas_return='single',
                      shots=num_shots)
real_job = real_armonk_backend.run(armonk_qobj)
job_id = real_job._job_id

In [83]:
def_armonk_qobj = assemble(default_schedule,
                      backend=real_armonk_backend,
                      qubit_lo_freq=[freq_est],
                      meas_level=2,
                      meas_return='single',
                      shots=num_shots)
def_real_job = real_armonk_backend.run(def_armonk_qobj)
def_job_id = def_real_job._job_id